In [ ]:
# READ SPAWN POINT
import json, os

fn = os.path.join(os.getcwd(),"research","eh21","b_40.spawnpoint")
schemafn = os.path.join(os.getcwd(),"kh2lib", "data", "schemas", "spawnpoint.json")

class SchemaReader:
    def __init__(self, fn, schemafn, includehidden=False):
        self.includehidden = includehidden
        self.data = open(fn, "rb").read()
        self.pos = 0
        self.schema = json.loads(open(schemafn).read())
        self.sp = {}
        self.sp["filename"] = os.path.basename(fn)
    def readBytes(self, nbytes):
        if self.pos+nbytes > len(self.data):
            raise Exception("Trying to read too much data")
        value = int(''.join([str(i) for i in self.data[self.pos:self.pos+nbytes][::-1]]))
        self.pos = self.pos + nbytes
        return value
    def addValue(self, sp, key, value, hidden=False):
        if hidden and not self.includehidden:
            return # We don't want this included in the object
        sp[key] = value
    def getLength(self, tpe):
        if tpe == "float" or "int":
            return 4
        if tpe == "short":
            return 2
        if tpe == "byte":
            return 1
    def read_by_schema(self, sp=None, schema=None):
        print("Parsing schema")
        if not sp:
            sp = self.sp # top level
        if not schema:
            schema = self.schema # top level
        for key in sorted([i for i in schema.keys() if type(schema[i]) != str], key=lambda k: schema[k]["seq"]):
            variable = schema[key]
            kind = variable["kind"]
            hidden = variable["hidden"] if "hidden" in variable else False
            if kind == "setvalue":
                setvalue = variable["value"]
                length = self.getLength(variable["type"])
                value = self.readBytes(length)
                if setvalue != value:
                    print("Warning: Value at pos {} should be {} but was read as {}".format(self.pos, setvalue, value))
                self.addValue(sp, key, value, hidden=hidden)
            elif kind == "value":
                length =  self.getLength(variable["type"])
                value = self.readBytes(length)
                self.addValue(sp, key, value, hidden=hidden)
            elif kind == "list":
                count = sp[variable["count_defined_by"]]
                self.addValue(sp, key, [], hidden=hidden)
                for _ in range(count):
                    sp[key].append(self.read_by_schema({}, variable["schema"]))
            elif kind.startswith("tuple"):
                count = kind.split('_')[1]
                length = self.getLength(variable["type"])
                t = []
                for _ in range(count):
                    value = self.readBytes(length)
                    t.append(value)
                self.addValue(sp, key, tuple(t), hidden=hidden)
            else:
                raise Exception("Error: Unknown how to handle variable {} with type {}".format(key, kind))

                
sr = SchemaReader(fn, schemafn, includehidden=True)
sr.read_by_schema()
sr.sp